著作権および使用上の注意：
このコードは、張沢立が大会のTABの仕事内容を軽減するため、試験的に作り出したものであり、大会のTABを担当するとき使用しても構わないが、商業的な利用は禁じます。また、このソフトを使用したことによるトラブル（メールが届かなかった～など）は一切責任負いかねませんので、ご注意ください。

バージョン1.0.2からの変更点


1.   web scrapingより、URLを自動に獲得のため、スプレッドシート「PURL送信用」に、送るURLを記入する必要がなくなった

2.   ★ログイン状態を保つために、**請求時にcookieをつけることに注意**、ここがさらに改善できるところかな★



バージョン1.0.3からの変更点


> 1.0.3に関しては、ログイン機能をつけずに、Cookieによりしかログインできないが、1.0.4では、TabのUsernameとPasswordを入力すれば、seleniumによるログイン、cookieの保存機能を付けました。



In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

In [ ]:
'''google driveにあるファイルを用いて、PURLの自動探索や、
ファイルへの書き込み、読み込みを実現するために、
google driveの認証を行い、driveにある（特にxlsx）ファイル
へのアクセスを可能にするためのセルです'''


# 認証のためのコード
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
'''driveにある、「PURL送信用」★ファイル名注意
というスプレッドシートを開き、そこから、送信先の名前
とメールを送るメールアドレスを取得し、それぞれリストにする
（互いのリストのindexが対応していることに注意）'''

## スプレッドシートを開く（名前から）
filename = "PURL送信用"
ss = gc.open(filename)

# データを取得する。（リストとして）
st = ss.worksheet("送信先")
allinfo = st.get_all_values()
allinfo.pop(0)

st2 = ss.worksheet("本文内容")
message = st2.get_all_values()
message=message[1][0]


#error防止の一環として、allinfoの内容を洗練する
for i in range(len(allinfo)):
  allinfo[i].pop() #まず空の第３項を取り除く
  for j in range(len(allinfo[i])):
    if allinfo[i][j] != "":
      while allinfo[i][j][-1] == " ":
        tem = list(allinfo[i][j])
        tem.pop()
        allinfo[i][j] = "".join(tem)

print(allinfo)

[['Kotaro Yoshida', 'ykotaro18@gmail.com'], ['Ryouta Ojima', 'grey.at.twilight7@gmail.com'], ['Minami Matsushima', 'minamidebate@gmail.com'], ['Natsumi Sadaoka', 'natsumi_sadaoka_7@keio.jp'], ['Yoshifumi Omura', 'omura.yoshifum@gmail.com'], ['Shoma Otsuka', 'shoma3408@gmail.com'], ['Tomoyuki Suda', 'sudakkuma@gmail.com'], ['Kyoka Kawakami', 'k.kawakami127@gmail.com'], ['Tasuku Arai', '19tasu96@gmail.com'], ['Xinyue Huang', 'anastasiahuangx@gmail.com'], ['So Ushiroda', 'ushirodaso@gmail.com'], ['Haruka Sato', 'sugarfar0319@gmail.com'], ['Satoshi Inoue', 'sato4inoue@hotmail.co.jp'], ['Yui Kitsukawa', 'yuikitsukawa@gmail.com'], ['Minato Kuba', 'minatokuba0314@yahoo.co.jp'], ['Mariko Kawabata', 'kawabata-mariko351@g.ecc.u-tokyo.ac.jp'], ['Kei Namba', 'numberkcollege@gmail.com'], ['Sota Mitsuhashi', 'sota3284@akane.waseda.jp'], ['Yukiko Watari', 'watari.yukiko.v0@tufs.ac.jp'], ['Ritsu Murata', 'lawyer08voice08@gmail.com'], ['Atsuki Ito', 'mmh-atsuki@g.ecc.u-tokyo.ac.jp'], ['Dohyun (Daniel) 

# ★この↓のコードを最初１回実行する必要があります。
# 前準備のダウンロードです！やらないとエラーが出ます！

In [ ]:
##★前準備のダウンロード★
!apt install fonts-ipafont-gothic
!pip install selenium
!apt update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-ipafont-mincho
The following NEW packages will be installed:
  fonts-ipafont-gothic fonts-ipafont-mincho
0 upgraded, 2 newly installed, 0 to remove and 76 not upgraded.
Need to get 8,251 kB of archives.
After this operation, 28.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 fonts-ipafont-gothic all 00303-18ubuntu1 [3,526 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 fonts-ipafont-mincho all 00303-18ubuntu1 [4,725 kB]
Fetched 8,251 kB in 1s (6,980 kB/s)
Selecting previously unselected package fonts-ipafont-gothic.
(Reading database ... 128422 files and directories currently installed.)
Preparing to unpack .../fonts-ipafont-gothic_00303-18ubuntu1_all.deb ...
Unpacking fonts-ipafont-gothic (00303-18ubuntu1) ...
Selecting previously unselected package fonts-ipaf

In [ ]:
"""★★★チェック事項
メールのタイトルがあっているか
メールの本文内容の大会名を変えたか
"""

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
import time
import requests
from lxml import etree

'''google driveにあるファイルを用いて、PURLの自動探索や、
ファイルへの書き込み、読み込みを実現するために、
google driveの認証を行い、driveにある（特にxlsx）ファイル
へのアクセスを可能にするためのセルです'''


# 認証のためのコード
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

'''driveにある、「PURL送信用」★ファイル名注意
というスプレッドシートを開き、そこから、送信先の名前
とメールを送るメールアドレスを取得し、それぞれリストにする
（互いのリストのindexが対応していることに注意）'''

## スプレッドシートを開く（名前から）
filename = "PURL送信用"
ss = gc.open(filename)

# データを取得する。（リストとして）
st = ss.worksheet("送信先")
allinfo = st.get_all_values()
allinfo.pop(0)

st2 = ss.worksheet("本文内容")
message = st2.get_all_values()
message=message[1][0]


#error防止の一環として、allinfoの内容を洗練する
for i in range(len(allinfo)):
  allinfo[i].pop() #まず空の第３項を取り除く
  for j in range(len(allinfo[i])):
    if allinfo[i][j] != "":
      while allinfo[i][j][-1] == " ":
        tem = list(allinfo[i][j])
        tem.pop()
        allinfo[i][j] = "".join(tem)

print(allinfo)

#webdeiverの設定
options = webdriver.ChromeOptions()
options.add_argument('--headless')#headless modeでchromeを起動する
options.add_argument('--no-sandbox')# sandboxを使わない
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options) #optionsを入れる
driver.implicitly_wait(10)# wait for 10s

#chromeをサーバー上でインストールすることができるが、headlessmodeでは入力ができないので、困った。
#何とか解決


def login(url):#
    username = input("Tabのusernameを入力してください") #usernameの入力
    password = input("Tabのpasswordを入力してください") #passwordの入力


    #driverの指定、指定時に自分のChromeのバージョンと一致するものを指定する
    #login_urlを獲得
    resp = requests.get(url)
    login_url = resp.url
    #ログインしたいURLを特定
    driver.get(login_url)#URLへのサクセス
    find_username = driver.find_element(By.NAME,"username")#usernameをいれる枠を探す
    find_password = driver.find_element(By.NAME,"password")#passwordいれる枠を探す
    find_username.send_keys(username)#input the username
    find_password.send_keys(password)# input the password
    login_button = driver.find_element(By.NAME,"continue")# find the login button
    login_button.click()#click the button
    cook = driver.get_cookies()
    return cook


#ログイン状態で使えるために、cookieの導入
#実際の請求
url = input("プライベートURLの画面から、ページのURLをコピーし、ここに貼り付けてください")
while True:
    try:
        cookie={"csrftoken":str(cook[1]["value"]), "sessionid":str(cook[0]["value"])}
        resp = requests.get(url,cookies=cookie)
        if "accounts/login" in resp.url:#loginが成功したかどうかを見ましょう
            login_url = resp.url
            cook = login(url)
        else:
            break
        print(resp.url)
    except Exception:
        cook = login(url)
    finally:
        pass

#以下ページからPURLを獲得する手順に入る
from lxml import etree #解析用のやつを導入
html = etree.HTML(resp.text)#decodeする
before_arrange = html.xpath("..//html//script[contains(text(),'privateurls')]/text()")#privateurlという文字列を含むものを全部抽出し、before_arrangeと名づける


#以下、獲得内容の中の有用な部分を抽出

arrange1 = str(before_arrange[0]).split('"') #上記冗長な文字列を、"で分割する

#人の名前とurlは必ず、textの２つ以降の要素となることに注意して、名前とＵＲＬを含むものを収集する
name_purl=[]
for i in range(len(arrange1)):
    if arrange1[i] == "text":
        name_purl.append(arrange1[i+2])

#上記方法で収集した余分のものをのぞく、それをnew_name_and_purlに保存する
new_name_and_purl = []
for i in range(len(name_purl)):
    if "View" in name_purl[i] or "Record" in name_purl[i] or ("ud83d" in name_purl[i] and "udd17" in name_purl[i] and not("https" in name_purl[i])):#余分のものを取り除く
        pass
    else:
        new_name_and_purl.append(name_purl[i])

#さらに、"https,privateurls"を含むかどうかを判断して、nameとpurlで両者を分ける
name = []
purl = []
for x in new_name_and_purl:
    if "https" in x and "privateurls" in x:
        purl.append(x)
    else:
        name.append(x)

#check the lenth of name and url to see if they are the same

if len(name) == len(purl):
    pass
else:
    print("要注意！どっか間違ってる")
    raise NameError

##取得したURLを、既定のスプレッドシートの欄に入れる
for i in range(len(allinfo)):
  if allinfo[i][0] in name:
    dex = name.index(allinfo[i][0])
    st.update_acell(f"C{i+2}",purl[dex])
    allinfo[i].append(purl[dex])
  else:
    print(allinfo[i][0],"と一致する名前がTAB上に見つかりませんでした")
    n = input("この方のprivate urlを入力してください")
    st.update(f"C{i+2}",n)
    allinfo[i].append(n)
print(allinfo)
#allinfoで獲得したPURLは本当に本人のものであるかをすべて確認する
checklist=[]
for x in allinfo:
  try:
    response = requests.get(x[2])
  except Exception:
    checklist.append(False)
  if response.status_code == 200:
    if f"for {x[0]}" in response.text:
      checklist.append(True)
    else:
      checklist.append(False)
  else:
    print(f"{allinfo[0]}のURLがアクセスできません、チェックしてください！")
    checklist.append(False)

[['Kotaro Yoshida', 'ykotaro18@gmail.com'], ['Ryouta Ojima', 'grey.at.twilight7@gmail.com'], ['Minami Matsushima', 'minamidebate@gmail.com'], ['Natsumi Sadaoka', 'natsumi_sadaoka_7@keio.jp'], ['Yoshifumi Omura', 'omura.yoshifum@gmail.com'], ['Shoma Otsuka', 'shoma3408@gmail.com'], ['Tomoyuki Suda', 'sudakkuma@gmail.com'], ['Kyoka Kawakami', 'k.kawakami127@gmail.com'], ['Tasuku Arai', '19tasu96@gmail.com'], ['Xinyue Huang', 'anastasiahuangx@gmail.com'], ['So Ushiroda', 'ushirodaso@gmail.com'], ['Haruka Sato', 'sugarfar0319@gmail.com'], ['Satoshi Inoue', 'sato4inoue@hotmail.co.jp'], ['Yui Kitsukawa', 'yuikitsukawa@gmail.com'], ['Minato Kuba', 'minatokuba0314@yahoo.co.jp'], ['Mariko Kawabata', 'kawabata-mariko351@g.ecc.u-tokyo.ac.jp'], ['Kei Namba', 'numberkcollege@gmail.com'], ['Sota Mitsuhashi', 'sota3284@akane.waseda.jp'], ['Yukiko Watari', 'watari.yukiko.v0@tufs.ac.jp'], ['Ritsu Murata', 'lawyer08voice08@gmail.com'], ['Atsuki Ito', 'mmh-atsuki@g.ecc.u-tokyo.ac.jp'], ['Dohyun (Daniel) 

In [ ]:
'''設定したリストの内容に基づき、送信'''


from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
import smtplib
from os.path import basename
from google.colab import drive
import random
import time



#greetinglist=["挨拶1","挨拶2","挨拶3","挨拶4"]

#パスをドライブにする
drive.mount("/content/gdrive")


#for loopで順番にメールを送る
#送信元のアドレス設定(gmail推奨)

# while True:
#   from_email = input("送信元のアドレスを入力してください")
#   n = input("上記アドレスで間違いなければ、[ok]を入力してください")
#   if n == "ok":
#     break

for x in allinfo:
  print(x)
  #greeting = greetinglist[random.randint(0,len(greetinglist)-1)]
  name =x[0]
  subject= "Privae URL for "+x[0]+"さん_KDO 2023"
  to_email = x[1]
  purl=x[2]
  final_text=message.format(name,purl)

  file=[]#添付ファイルのパスのリスト


  # SMTP認証情報
  account = "zhangzeli004@gmail.com"
  password = "msyetwsmosecldcw"
  from_email = "zhangzeli004@gmail.com"

  # 送受信先
  #to_email = "送信先@hoge.com"
  #from_email = "送信元@gmail.com"


  # MIMEの作成
  msg = MIMEMultipart()
  msg["Subject"] = subject
  msg["To"] = to_email
  msg["From"] = from_email
  msg.attach(MIMEText(final_text))


  # ファイルを添付
  if len(file)!= 0:
    for path in file:
      with open(path, "rb") as f:
          part = MIMEApplication(
              f.read(),
              Name=basename(path)
          )

      part['Content-Disposition'] = 'attachment; filename="%s"' % basename(path)
      msg.attach(part)

  # メール送信処理
  server = smtplib.SMTP("smtp.gmail.com", 587)
  server.starttls()
  server.login(account, password)

  print(final_text,"\n")
  print(name,to_email,"へ以上内容をx秒後送信します")
  time.sleep(5)
  server.send_message(msg)


  server.quit()